# 2025 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

In [57]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as func
from torch.utils.data import DataLoader,Dataset
from transformers import RobertaTokenizer, RobertaModel
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
import os
path = '/content/drive/MyDrive/Automated-Fact-checking-System'
os.chdir(path)
os.listdir(path)


['Project Group.md',
 'eval.py',
 'README.md',
 '.gitignore',
 'COMP90042_Project_2025.ipynb',
 'data',
 '.git']

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [60]:
class ClaimEvidenceDataset(Dataset):

    def __init__(self, claims, evidence, tokenizer, max_len=512, if_train=True):
        self.claim_texts = []
        self.evidence_texts = []
        self.tokenizer = tokenizer
        self.max_length = max_len
        self.if_train = if_train
        if if_train:
            self.labels = []

        self.label2id = {
            "SUPPORTS": 0,
            "REFUTES": 1,
            "NOT_ENOUGH_INFO": 2,
            "DISPUTED": 3
        }

        for claim_idx, claim_info in claims.items():
            self.claim_texts.append(claim_info['claim_text']+'</s>')
            evidence_text = ""
            for evid in claim_info['evidences']:
                if evid in evidence:
                    evidence_text += evidence[evid] + " "
            self.evidence_texts.append(evidence_text.strip())
            if if_train:
                self.labels.append(self.label2id[claim_info['claim_label']])

    def __len__(self):
        return len(self.claim_texts)

    def __getitem__(self, idx):
        encoded = self.tokenizer(
            self.claim_texts[idx],
            self.evidence_texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        if self.if_train:
            return {
                'input_ids': encoded['input_ids'].squeeze(0),
                'attention_mask': encoded['attention_mask'].squeeze(0),
                'labels': torch.tensor(self.labels[idx], dtype=torch.int)
            }
        else:
            return {
                'input_ids': encoded['input_ids'].squeeze(0),
                'attention_mask': encoded['attention_mask'].squeeze(0),
            }


In [61]:
with open("data/dev-claims.json", 'r') as f:
    claims = json.load(f)

with open("data/evidence.json", 'r') as f:
    evidence = json.load(f)

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

dataset = ClaimEvidenceDataset(claims, evidence, tokenizer)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*